### Задание №6

Задание состоит в том, чтобы реализовать базовые алгоритмы линейной алгебры с помощью базовых средств объектно-ориентированного программирования Python.

Для решения задачи требуется реализовать класс Matrix с базовыми алгоритмами линейной алгебры:

\_\_mul\_\_ - умножение матриц

\_\_add\_\_ - поэлементное сложение матриц

\_\_sub\_\_ - поэлементное вычитание матриц

\_\_pow\_\_ - возведение элементов матрицы в скаларную степень

transpose() - транспонирование матрицы

inverse() - обращение матрицы

det() - определитель матрицы

shape() - размер матрицы (кортеж)

sum() - сумма всех элементов матрицы

а также служебные методы:

\_\_getitem\_\_ - получить элемент по индексу

\_\_setitem\_\_ - задать элемент по индексу

\_\_repr\_\_ и \_\_str\_\_

### Отправка задания

Для сдачи задания необходимо отправить боту @py2022sharebot с указанием seminar06_1 два файла:
1. result.json (файл, создаваемый в последней ячейке)
2. seminar06_1.ipynb (этот ноутбук)

Автоматическая проверка отправки будет реализована командой /check seminar06_1.

In [1]:
import sys
import os
import math
import copy
import json
import random
import numpy as np

Требуется реализовать методы в следующем классе, отмеченные #TODO:

In [2]:
class Matrix:
    def __init__(self, nrows, ncols, init="zeros"):
        """Конструктор класса Matrix.
        Создаёт матрицу резмера nrows x ncols и инициализирует её методом init.
        nrows - количество строк матрицы
        ncols - количество столбцов матрицы
        init - метод инициализации элементов матрицы:
            "zeros" - инициализация нулями
            "ones" - инициализация единицами
            "random" - случайная инициализация
            "eye" - матрица с единицами на главной диагонали
        """
        
        #TODO При отрицательном значении nrows или ncols - ValueError
        if (nrows <= 0) | (ncols <= 0):
            raise ValueError('Positive integer expected')
            
        #TODO При методе init, отличном от "zeros", "ones", "eye" и "random" - ValueError
        if (init != "zeros") & (init != "ones") & (init != "random") & (init != "eye"):
            raise ValueError('Wrong init method')
            
        self.nrows = nrows
        self.ncols = ncols
        self.init = init

        #TODO инициализировать self.data
        if init == "zeros":
            self.data = [[0 for _ in range(self.ncols)] for _ in range(self.nrows)]
        elif init == "ones":
            self.data = [[1 for _ in range(self.ncols)] for _ in range(self.nrows)]
        elif init == "random":
            self.data = [[random.random() for _ in range(self.ncols)] for _ in range(self.nrows)]
        else:
            self.data = [[1 if _ == row else 0 for _ in range(self.ncols)] for row in range(self.nrows)]
    
    @staticmethod
    def from_dict(data):
        "Десериализация матрицы из словаря"
        ncols = data["ncols"]
        nrows = data["nrows"]
        items = data["data"]
        assert len(items) == ncols*nrows
        result = Matrix(nrows, ncols)
        for row in range(nrows):
            for col in range(ncols):
                result[(row, col)] = items[ncols*row + col]
        return result
    
    @staticmethod
    def to_dict(matr):
        "Сериализация матрицы в словарь"
        assert isinstance(matr, Matrix)
        nrows, ncols = matr.shape()
        data = []
        for row in range(nrows):
            for col in range(ncols):
                data.append(matr[(row, col)])
        return {"nrows": nrows, "ncols": ncols, "data": data}
    
    def __str__(self):
        #TODO имплементировать метод
        return str(self.data)
    
    def __repr__(self):
        #TODO имплементировать метод
        return f'Matrix(nrows={self.nrows}, ncols={self.ncols}, init={self.init})'
    
    def shape(self):
        "Вернуть кортеж размера матрицы (nrows, ncols)"
        return self.nrows, self.ncols
    
    def __getitem__(self, index):
        """Получить элемент матрицы по индексу index
        index - список или кортеж, содержащий два элемента
        """
        #TODO Если index - не кортеж или список, и он не содержит два элемента - ValueError
        #TODO Если index за пределами размера матрицы - IndexError
        if (type(index) != list) & (type(index) != tuple):
            raise ValueError('index must be list or tuple')
        elif len(index) != 2:
            raise ValueError(f'index must contain two elements, got {len(index)}')
        elif (not 0 <= index[0] < self.nrows) or (not 0 <= index[1] < self.ncols):
            raise IndexError('index out of data')
        
        row, col = index
        #TODO имплементировать метод
        return self.data[row][col]
    
    def __setitem__(self, index, value):
        """Задать элемент матрицы по индексу index
        index - список или кортеж, содержащий два элемента
        value - Устанавливаемое значение
        """
        #TODO Если index - не кортеж или список, и он не содержит два элемента - ValueError
        #TODO Если index за пределами размера матрицы - IndexError
        if (type(index) != list) & (type(index) != tuple):
            raise ValueError('index must be list or tuple')
        elif len(index) != 2:
            raise ValueError(f'index must contain two elements, got {len(index)}')
        elif (not 0 <= index[0] < self.nrows) or (not 0 <= index[1] < self.ncols):
            raise IndexError('index out of data')
            
        row, col = index
        #TODO имплементировать метод
        self.data[row][col] = value
    
    def __sub__(self, rhs):
        "Вычесть матрицу rhs и вернуть результат"
        #TODO Если размер rhs отличается от размера данной матрицы - ValueError
        if self.shape() != rhs.shape():
            raise ValueError
        #TODO имплементировать метод
        nrows, ncols = self.shape()
        sub = Matrix(nrows, ncols)
        for i in range(nrows):
            for j in range(ncols):
                sub.__setitem__([i, j], (self.__getitem__([i,j]) - rhs.__getitem__([i,j])))
        return sub
        
    
    def __add__(self, rhs):
        "Сложить с матрицей rhs и вернуть результат"
        #TODO Если размер rhs отличается от размера данной матрицы - ValueError
        if self.shape() != rhs.shape():
            raise ValueError
        #TODO имплементировать метод
        nrows, ncols = self.shape()
        add = Matrix(nrows, ncols)
        for i in range(nrows):
            for j in range(ncols):
                add.__setitem__([i, j], (self.__getitem__([i,j]) + rhs.__getitem__([i,j])))
        return add
    
    def __mul__(self, rhs):
        "Умножить на матрицу rhs и вернуть результат"
        #TODO Если число строк rhs отличается от числа столбцов данной матрицы - ValueError
        if self.shape()[1] != rhs.shape()[0]:
            raise ValueError
        #TODO имплементировать метод
        nrows_self, ncols_self = self.shape()
        ncols_rhs = rhs.shape()[1]
        mul = Matrix(nrows_self, ncols_rhs)
        for row in range(nrows_self):
            for col in range(ncols_rhs):
                s = 0
                for i in range(ncols_self):
                    s += self.__getitem__([row, i])*rhs.__getitem__([i, col])
                mul.__setitem__([row, col], s)
        return mul
    
    def __pow__(self, power):
        "Возвести все элементы в степень power и вернуть результат"
        #TODO имплементировать метод
        nrows, ncols = self.shape()
        pow_mat = Matrix(nrows, ncols)
        for i in range(nrows):
            for j in range(ncols):
                pow_mat.__setitem__([i,j], self.__getitem__([i,j])**power)
        return pow_mat
    
    def sum(self):
        "Вернуть сумму всех элементов матрицы"
        #TODO имплементировать метод
        nrows, ncols = self.shape()
        s = 0
        for i in range(nrows):
            for j in range(ncols):
                s += self.__getitem__([i,j])
        return s
        
    def det(self):
        "Вычислить определитель матрицы"
        #TODO имплементировать метод
        det = 0
        nrows, ncols = self.shape()
        if nrows == 2 and ncols == 2:
            det = self.__getitem__([0,0]) * self.__getitem__([1,1]) - self.__getitem__([1,0]) * self.__getitem__([0,1])
            return det 
        for col in range(ncols):
            alg_dop_matr = self.data[1:].copy()
            for row in range(nrows - 1):
                alg_dop_matr[row] = alg_dop_matr[row][:col] + alg_dop_matr[row][(col+1):]
            dop_matr = Matrix(nrows - 1, ncols - 1)
            dop_matr.data = alg_dop_matr
            alg_dop_det = dop_matr.det()
            det += ((-1)**col) * self.__getitem__([0, col]) * alg_dop_det
        return det
    
    def transpose(self):
        "Транспонировать матрицу и вернуть результат"
        #TODO имплементировать метод
        nrows_self, ncols_self = self.shape()
        transpose = Matrix(nrows = ncols_self, ncols = nrows_self)
        for i in range(ncols_self):
            for j in range(nrows_self):
                transpose.__setitem__([i,j], self.__getitem__([j,i]))
        return transpose
    
    def inv(self):
        "Вычислить обратную матрицу и вернуть результат"
        nrows, ncols = self.shape()
        #TODO Если матрица не квадратная - ArithmeticError
        if ncols != nrows:
            raise ArithmeticError
        #TODO Если определитель равен нулю - ArithmeticError
        determ = self.det()
        if determ == 0:
            raise ArithmeticError
        #TODO имплементировать метод
        inv = Matrix(nrows = ncols, ncols = nrows)
        for i in range(nrows):
            for j in range(ncols):
                alg_dop_matr = self.data[:i] + self.data[(i+1):]
                for row in range(len(alg_dop_matr)):
                    alg_dop_matr[row] = alg_dop_matr[row][:j] + alg_dop_matr[row][(j+1):]
                dop_matr = Matrix(nrows - 1, ncols - 1)
                dop_matr.data = alg_dop_matr
                inv.__setitem__([j,i], (((-1)**(i+j)) * dop_matr.det())/determ)
        return inv

    def tonumpy(self):
        "Приведение к массиву numpy"
        #TODO имплементировать метод
        np_data = np.array(self.data)
        return np_data

In [3]:
def load_file(filename):
    with open(filename, "r") as f:
        input_file = json.load(f)
        A = Matrix.from_dict(input_file["A"])
        B = Matrix.from_dict(input_file["B"])
    return A, B

Проверка реализованных методов

In [4]:
# Задайте в filename путь к файлу, полученному от бота
filename = "input_008.json"
A, B = load_file(filename)
print("Матрица A: ")
print(A)
print("Матрица B: ")
print(B)
C = A*B
print("Матрица C = A*B: ")
print(C)
C_t = C.transpose()
print("Транспонированная матрица C: ")
print(C_t)
C_inv = C.inv()
print("Матрица, обратная C: ")
print(C_inv)
E = Matrix(C_inv.ncols, C_inv.nrows, init="eye")
D = C_inv + E
print("Матрица D равная сумме C и единичной матрицы: ")
print(D)
D_det = D.det()
print("Определитель матрицы D: ", D_det)
D_norm = (D**2).sum()**0.5
print("Норма Фробениуса матрицы D: ", D_norm)

Матрица A: 
[[0.9911885424248187, 0.30763913963976297, 0.07678670791974618, 0.35417951338817155], [0.6641255576513478, 0.05944048395423007, 0.12874901048932452, 0.8174188458343974], [0.2079009692031285, 0.8579119067187833, 0.9954692265788966, 0.8704571755096633]]
Матрица B: 
[[0.2887112332332733, 0.06445300813805166, 0.6634380209162374], [0.6169079101680625, 0.7715342190931229, 0.5132207257174112], [0.07409306457551801, 0.6615288764857252, 0.7115221742601935], [0.9613144387575379, 0.7731535254680307, 0.15244174682242073]]
Матрица C = A*B: 
[[0.8221195278118343, 0.6258709705994987, 0.9241061365291188], [1.023745761268186, 0.8058267080406594, 0.6873287665418621], [1.499816403516605, 2.0068369089100186, 1.4192200197958464]]
Транспонированная матрица C: 
[[0.8221195278118343, 1.023745761268186, 1.499816403516605], [0.6258709705994987, 0.8058267080406594, 2.0068369089100186], [0.9241061365291188, 0.6873287665418621, 1.4192200197958464]]
Матрица, обратная C: 
[[-0.7280294801947064, 2.9845045

Сохранение результатов в файл. Не изменяйте этот код. Отправтье файл result.json в ответ на это задание.

In [5]:
A_dict = Matrix.to_dict(A)
B_dict = Matrix.to_dict(B)
C_dict = Matrix.to_dict(C)
Ct_dict = Matrix.to_dict(C_t)
Cinv_dict = Matrix.to_dict(C_inv)
result = {
    "A": A_dict,
    "B": B_dict,
    "C": C_dict,
    "Ct": Ct_dict,
    "Cinv": Cinv_dict,
    "D_det": D_det,
    "D_norm": D_norm
}

In [6]:
def save_file(filename, data):
    with open(filename, "w") as f:
        input_file = json.dump(data, f)

In [7]:
save_file("result.json", result)

In [31]:
a_mat = Matrix(nrows = random.randint(2, 10), ncols = random.randint(2, 10), init = 'random')
b_mat = Matrix(nrows = a_mat.nrows, ncols = a_mat.ncols, init = 'random')
a_np = a_mat.tonumpy()
b_np = b_mat.tonumpy()
c_mat = copy.copy(a_mat).transpose() * b_mat
c_np = np.dot(a_np.T,b_np)
print('frobenius_norm_for_mul_error = {}'.format(np.linalg.norm(c_mat.tonumpy() - c_np)))
print('frobenius_norm_for_add_error = {}'.format(np.linalg.norm((a_mat+b_mat).tonumpy() - (a_np+b_np))))
print('frobenius_norm_for_sub_error = {}'.format(np.linalg.norm((a_mat-b_mat).tonumpy() - (a_np-b_np))))
print('frobenius_norm_for_inv_error = {}'.format(np.linalg.norm(c_mat.inv().tonumpy() - np.linalg.inv(c_np))))
print('|det_mat - det_np| = {}'.format(abs(c_mat.det() - np.linalg.det(c_np))))
print('|sum_mat - sum_np| = {}'.format(abs(a_mat.sum() - np.sum(a_np))))
if not np.array_equal(a_mat.shape(), np.shape(a_np)):
    print('shape_error!')
else:
    print('shapes ok')

frobenius_norm_for_mul_error = 1.041481514324134e-15
frobenius_norm_for_add_error = 0.0
frobenius_norm_for_sub_error = 0.0
frobenius_norm_for_inv_error = 1.4472784908161385e-11
|det_mat - det_np| = 1.2880321809127793e-16
|sum_mat - sum_np| = 3.552713678800501e-15
shapes ok
